<a href="https://colab.research.google.com/github/DeepLearningSaeid/Grad/blob/main/Optimized%20SWAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install autokeras -q





     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.6/148.6 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.5/584.5 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 111.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 42.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 60.4 MB/s eta 0:00:00


In [19]:
import autokeras as ak
import numpy as np
import timeit
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.models import Model, Sequential
from keras.layers import Input, Embedding, Dense, concatenate, Dropout, Flatten, Activation
from keras import backend as K
from keras.utils import get_custom_objects, to_categorical, plot_model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import GridSearchCV, KFold, train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.datasets import load_iris, load_digits, load_wine, load_diabetes, load_breast_cancer, fetch_olivetti_faces, load_linnerud
from tensorflow.keras.datasets import mnist, fashion_mnist
from sklearn.datasets import load_digits

In [13]:


# Set random seed
np.random.seed(110)

# Define custom activation functions and update the custom objects dictionary
def define_activation_functions():
    """
    Define custom activation functions and update the custom objects dictionary.
    """
    # Define activation functions and their respective names
    activation_functions = [
        ('X_1', lambda x: tf.pow(x, 1)),
        ('X_2', lambda x: tf.pow(x, 2) / 2),
        ('X_2_', lambda x: tf.pow(x, 2) / 24),
        ('X_2__', lambda x: tf.pow(x, 2) / 720),
        ('X_2___', lambda x: tf.pow(x, 2) / 40320),
    ]

    # Update the custom objects dictionary with the defined activation functions
    for name, function in activation_functions:
        get_custom_objects().update({name: Activation(function)})

# Define and register the custom activation functions
define_activation_functions()

def create_optimized_model(input_dim, hidden_dim, output_dim, metrics='accuracy', learning_rate=0.001):
    """
    Create an optimized SWAG model with a custom architecture.

    :param input_dim: int, dimension of the input data
    :param output_dim: int, dimension of the output data
    :param hidden_dim: int, hidden layer dimension, default is 50
    :return: Model, a compiled Keras model
    """

    # Define input layer
    input_layer = Input(shape=(input_dim,))

    # First layer with custom activations
    layer_1_x1 = Dense(hidden_dim, activation='X_1')(input_layer)
    layer_1_x2 = Dense(hidden_dim, activation='X_2')(input_layer)
    concat_first_layer = concatenate([layer_1_x1, layer_1_x2])

    # Second layer with custom activations
    layer_x3_x4 = Dense(hidden_dim, activation='X_2_')(concat_first_layer)

    # Third layer with custom activations
    layer_x5_x6 = Dense(hidden_dim, activation='X_2__')(layer_x3_x4)

    # Concatenate all layers
    concat_second_layer = concatenate([layer_1_x1, layer_1_x2, concat_first_layer,
                                       layer_x3_x4, layer_x5_x6])

    # Output layer for the concatenated layers
    output_first_layer = Dense(hidden_dim, activation='linear')(concat_second_layer)

    # Final output layer
    output_layer = Dense(output_dim, activation='linear')(output_first_layer)

    # Create and compile the model
    model = Model(input_layer, output_layer)

    # Define the optimizer
    opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)

    # Compile the model with specified loss and metrics
    model.compile(loss='mean_squared_error', optimizer=opt, metrics=[metrics])

    return model


In [14]:
# optimized_model = create_optimized_model(784,784, 10)

# Load and preprocess MNIST data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = (x_train + 10) / 300.0, (x_test + 10) / 300.0
x_train = x_train.reshape(-1, 784)
x_test = x_test.reshape(-1, 784)
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Create and train the model
optimized_model = create_optimized_model(784, 500, 10 , 'accuracy')
batch_size = 100
epochs = 2
start = timeit.default_timer()
history = optimized_model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(x_test, y_test))
end = timeit.default_timer()

# Training time
print(f"Training time: {end - start} seconds")

Epoch 1/2
600/600 [==============================] - 42s 68ms/step - loss: 0.0417 - accuracy: 0.8388 - val_loss: 0.0262 - val_accuracy: 0.9068
Epoch 2/2
600/600 [==============================] - 41s 69ms/step - loss: 0.0209 - accuracy: 0.9345 - val_loss: 0.0201 - val_accuracy: 0.9302
Training time: 143.77909544299996 seconds


In [20]:
# Load the Fashion MNIST dataset
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

x_train, x_test = (x_train + 10) / 300.0, (x_test + 10) / 300.0
x_train = x_train.reshape(-1, 784)
x_test = x_test.reshape(-1, 784)
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Create and train the model
optimized_model = create_optimized_model(784, 500, 10 , 'accuracy')
batch_size = 100
epochs = 2
start = timeit.default_timer()
history = optimized_model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(x_test, y_test))
end = timeit.default_timer()

# Training time
print(f"Training time: {end - start} seconds")

4422102/4422102 [==============================] - 0s 0us/step
Epoch 1/2
600/600 [==============================] - ETA: 0s - loss: 0.1130 - accuracy: 0.7804

600/600 [==============================] - 39s 63ms/step - loss: 0.1130 - accuracy: 0.7804 - val_loss: 0.0329 - val_accuracy: 0.8299
Epoch 2/2
600/600 [==============================] - 37s 62ms/step - loss: 0.0298 - accuracy: 0.8534 - val_loss: 0.0301 - val_accuracy: 0.8443
Training time: 83.51366655300012 seconds


In [16]:


iris = load_iris()
X = iris.data
y = iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

y_train = to_categorical(y_train, 3)
y_test = to_categorical(y_test, 3)


num_features = X_train.shape[1]
num_classes = y_train.shape[1]





# AutoKeras model
ak_model = ak.StructuredDataClassifier(max_trials=3, overwrite=True) # Set max_trials to an appropriate number

start_ak = timeit.default_timer()
ak_model.fit(X_train, y_train, epochs=10)
end_ak = timeit.default_timer()
print(f"AutoKeras Training time: {end_ak - start_ak} seconds")

# Evaluate AutoKeras model
_, ak_accuracy = ak_model.evaluate(X_test, y_test)
print(f"AutoKeras Model Accuracy: {ak_accuracy}")

print('******************************************************************************************')
print('******************************************************************************************')
print('******************************************************************************************')
print('******************************************************************************************')
print('******************************************************************************************')


# Create and train the model
model = create_optimized_model(num_features, 10, num_classes, 'accuracy')
batch_size = 10
epochs = 10
start = timeit.default_timer()
history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(X_test, y_test))
end = timeit.default_timer()

# Training time
print(f"Training time: {end - start} seconds")

# Evaluate your model
_, accuracy = model.evaluate(X_test, y_test)
print(f"SWAG Model Accuracy: {accuracy}")

Trial 3 Complete [00h 00m 02s]
val_accuracy: 0.7083333134651184

Best val_accuracy So Far: 0.9166666865348816
Total elapsed time: 00h 00m 07s
Epoch 1/10
4/4 [==============================] - 1s 4ms/step - loss: 1.1562 - accuracy: 0.3750
Epoch 2/10
4/4 [==============================] - 0s 3ms/step - loss: 1.0930 - accuracy: 0.4500
Epoch 3/10
4/4 [==============================] - 0s 4ms/step - loss: 1.0367 - accuracy: 0.5167
Epoch 4/10
4/4 [==============================] - 0s 3ms/step - loss: 0.9844 - accuracy: 0.5667
Epoch 5/10
4/4 [==============================] - 0s 3ms/step - loss: 0.9356 - accuracy: 0.5833
Epoch 6/10
4/4 [==============================] - 0s 3ms/step - loss: 0.8889 - accuracy: 0.6333
Epoch 7/10
4/4 [==============================] - 0s 3ms/step - loss: 0.8439 - accuracy: 0.6917
Epoch 8/10
4/4 [==============================] - 0s 3ms/step - loss: 0.8009 - accuracy: 0.7667
Epoch 9/10
4/4 [==============================] - 0s 3ms/step - loss: 0.7595 - accuracy: 0

In [17]:




# Load and preprocess the dataset
digits = load_digits()
X = digits.data
y = to_categorical(digits.target)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Using MinMaxScaler to scale the data between 0 and 1
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

num_features = X_train.shape[1]
num_classes = y_train.shape[1]

print(num_features)


# AutoKeras model
ak_model = ak.StructuredDataClassifier(max_trials=5, overwrite=True)

# Train the AutoKeras model
ak_model.fit(X_train, y_train, epochs=epochs)

# Evaluate AutoKeras model
loss, ak_accuracy = ak_model.evaluate(X_test, y_test)
print(f"AutoKeras Model Accuracy: {ak_accuracy}")

print('******************************************************************************************')
print('******************************************************************************************')
print('******************************************************************************************')
print('******************************************************************************************')
print('******************************************************************************************')

# Create and train the model
model = create_optimized_model(num_features, 200, num_classes, 'accuracy')
batch_size = 10
epochs = 5

# Train the model
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=1)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"SWAG Accuracy: {accuracy}")


Trial 5 Complete [00h 00m 16s]
val_accuracy: 0.7824561595916748

Best val_accuracy So Far: 0.8070175647735596
Total elapsed time: 00h 01m 21s
Epoch 1/10
45/45 [==============================] - 1s 4ms/step - loss: 2.2017 - accuracy: 0.2317
Epoch 2/10
45/45 [==============================] - 0s 3ms/step - loss: 1.6601 - accuracy: 0.5428
Epoch 3/10
45/45 [==============================] - 0s 3ms/step - loss: 1.1857 - accuracy: 0.6701
Epoch 4/10
45/45 [==============================] - 0s 3ms/step - loss: 0.8611 - accuracy: 0.7662
Epoch 5/10
45/45 [==============================] - 0s 4ms/step - loss: 0.6734 - accuracy: 0.8031
Epoch 6/10
45/45 [==============================] - 0s 3ms/step - loss: 0.5548 - accuracy: 0.8441
Epoch 7/10
45/45 [==============================] - 0s 3ms/step - loss: 0.4709 - accuracy: 0.8629
Epoch 8/10
45/45 [==============================] - 0s 3ms/step - loss: 0.4072 - accuracy: 0.8831
Epoch 9/10
45/45 [==============================] - 0s 4ms/step - loss: 0.

In [18]:


# Load and preprocess the dataset
wine = load_wine()
X = wine.data
y = to_categorical(wine.target)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Using MinMaxScaler to scale the data between 0 and 1
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

num_features = X_train.shape[1]
num_classes = y_train.shape[1]

print(num_features)


batch_size = 10
epochs = 5


# AutoKeras model
ak_model = ak.StructuredDataClassifier(max_trials=5, overwrite=True)

# Train the AutoKeras model
ak_model.fit(X_train, y_train, epochs=epochs)

# Evaluate AutoKeras model
loss, ak_accuracy = ak_model.evaluate(X_test, y_test)
print(f"AutoKeras Model Accuracy: {ak_accuracy}")

print('******************************************************************************************')
print('******************************************************************************************')
print('******************************************************************************************')
print('******************************************************************************************')
print('******************************************************************************************')


# Create and train the model
model = create_optimized_model(num_features, 20, num_classes, 'accuracy')


# Train the model
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=1)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"SWAG Accuracy: {accuracy}")

Trial 5 Complete [00h 00m 03s]
val_accuracy: 0.8571428656578064

Best val_accuracy So Far: 0.9285714030265808
Total elapsed time: 00h 00m 11s
Epoch 1/5
5/5 [==============================] - 1s 5ms/step - loss: 1.2534 - accuracy: 0.2887
Epoch 2/5
5/5 [==============================] - 0s 5ms/step - loss: 1.0856 - accuracy: 0.4577
Epoch 3/5
5/5 [==============================] - 0s 4ms/step - loss: 1.0563 - accuracy: 0.4648
Epoch 4/5
5/5 [==============================] - 0s 4ms/step - loss: 0.9120 - accuracy: 0.5704
Epoch 5/5
2/2 [==============================] - 0s 8ms/step - loss: 0.8572 - accuracy: 0.7778
AutoKeras Model Accuracy: 0.7777777910232544
******************************************************************************************
******************************************************************************************
******************************************************************************************
*****************************************************************

In [21]:


# Load and preprocess the dataset
cancer = load_breast_cancer()
X = cancer.data
y = cancer.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Using MinMaxScaler to scale the data between 0 and 1
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

num_features = X_train.shape[1]
# num_classes = y_train.shape[1]

print(num_features)
batch_size = 10
epochs = 5

# AutoKeras model
ak_model = ak.StructuredDataClassifier(max_trials=5, overwrite=True)

# Train the AutoKeras model
ak_model.fit(X_train, y_train, epochs=epochs)

# Evaluate AutoKeras model
loss, ak_accuracy = ak_model.evaluate(X_test, y_test)
print(f"AutoKeras Model Accuracy: {ak_accuracy}")

print('******************************************************************************************')
print('******************************************************************************************')
print('******************************************************************************************')
print('******************************************************************************************')
print('******************************************************************************************')


# Create and train the model
model = create_optimized_model(num_features, 20, 1, 'accuracy')


# Train the model
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=1)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Accuracy: {accuracy}")


Trial 5 Complete [00h 00m 03s]
val_accuracy: 0.47887325286865234

Best val_accuracy So Far: 0.98591548204422
Total elapsed time: 00h 00m 15s
Epoch 1/5
15/15 [==============================] - 1s 4ms/step - loss: 0.4757 - accuracy: 0.8703
Epoch 2/5
15/15 [==============================] - 0s 4ms/step - loss: 0.2101 - accuracy: 0.9495
Epoch 3/5
15/15 [==============================] - 0s 3ms/step - loss: 0.1292 - accuracy: 0.9604
Epoch 4/5
15/15 [==============================] - 0s 3ms/step - loss: 0.0959 - accuracy: 0.9758
Epoch 5/5
4/4 [==============================] - 0s 4ms/step - loss: 0.0780 - accuracy: 0.9649
AutoKeras Model Accuracy: 0.9649122953414917
******************************************************************************************
******************************************************************************************
******************************************************************************************
**********************************************************

In [22]:




# Load Boston Housing data
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.boston_housing.load_data(
    path="boston_housing.npz", test_split=0.2, seed=113
)

# Using MinMaxScaler to scale the data between 0 and 1
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

batch_size = 5
epochs = 30

# AutoKeras model
ak_model = ak.StructuredDataRegressor(max_trials=3, overwrite=True)

start_ak = timeit.default_timer()
ak_model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size)
end_ak = timeit.default_timer()
print(f"AutoKeras Training time: {end_ak - start_ak} seconds")

# Evaluate AutoKeras model
loss, ak_mae = ak_model.evaluate(X_test, y_test)
print(f"AutoKeras Model MAE: {ak_mae}")

print('******************************************************************************************')
print('******************************************************************************************')
print('******************************************************************************************')
print('******************************************************************************************')
print('******************************************************************************************')

# Create and train the model
input_dim = X_train.shape[1]
optimized_model = create_optimized_model(input_dim, input_dim , 1 , 'mae') # input_dim and hidden_dim

start = timeit.default_timer()
history = optimized_model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(X_test, y_test))
end = timeit.default_timer()

# Training time
print(f"Training time: {end - start} seconds")


Trial 3 Complete [00h 00m 06s]
val_loss: 12.89281177520752

Best val_loss So Far: 12.217472076416016
Total elapsed time: 00h 00m 26s
Epoch 1/30
81/81 [==============================] - 1s 2ms/step - loss: 262.0479 - mean_squared_error: 262.0479
Epoch 2/30
81/81 [==============================] - 0s 2ms/step - loss: 24.0782 - mean_squared_error: 24.0782
Epoch 3/30
81/81 [==============================] - 0s 2ms/step - loss: 16.5472 - mean_squared_error: 16.5472
Epoch 4/30
81/81 [==============================] - 0s 2ms/step - loss: 13.8998 - mean_squared_error: 13.8998
Epoch 5/30
81/81 [==============================] - 0s 3ms/step - loss: 12.2794 - mean_squared_error: 12.2794
Epoch 6/30
81/81 [==============================] - 0s 3ms/step - loss: 11.2639 - mean_squared_error: 11.2639
Epoch 7/30
81/81 [==============================] - 0s 3ms/step - loss: 10.5905 - mean_squared_error: 10.5905
Epoch 8/30
81/81 [==============================] - 0s 3ms/step - loss: 10.1265 - mean_squared_

In [24]:



# Load the dataset
diabetes = load_diabetes()
X = diabetes.data
y = diabetes.target

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features to be between 0 and 1
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

num_features = X_train.shape[1]
# num_classes = y_train.shape[1]

print(num_features)

batch_size = 10
epochs = 30
# AutoKeras model
ak_model = ak.StructuredDataRegressor(max_trials=3, overwrite=True)

# Train the AutoKeras model
ak_model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size)

# Evaluate AutoKeras model
loss_ak, ak_mae = ak_model.evaluate(X_test, y_test)
print(f"AutoKeras Model MAE: {ak_mae}")


print('******************************************************************************************')
print('******************************************************************************************')
print('******************************************************************************************')
print('******************************************************************************************')
print('******************************************************************************************')

# Create and train the model
model = create_optimized_model(num_features, 20, 1, 'mae')


# Train the model
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=1)

# Evaluate the model
loss, mae = model.evaluate(X_test, y_test)
print(f"SWAG Model MAE: {mae}")






Trial 3 Complete [00h 00m 09s]
val_loss: 3579.044189453125

Best val_loss So Far: 2992.968505859375
Total elapsed time: 00h 00m 23s
Epoch 1/30
36/36 [==============================] - 1s 2ms/step - loss: 29118.2246 - mean_squared_error: 29118.2246
Epoch 2/30
36/36 [==============================] - 0s 2ms/step - loss: 27349.4844 - mean_squared_error: 27349.4844
Epoch 3/30
36/36 [==============================] - 0s 2ms/step - loss: 23291.1250 - mean_squared_error: 23291.1250
Epoch 4/30
36/36 [==============================] - 0s 2ms/step - loss: 16400.8848 - mean_squared_error: 16400.8848
Epoch 5/30
36/36 [==============================] - 0s 2ms/step - loss: 9049.7822 - mean_squared_error: 9049.7822
Epoch 6/30
36/36 [==============================] - 0s 2ms/step - loss: 5199.1216 - mean_squared_error: 5199.1216
Epoch 7/30
36/36 [==============================] - 0s 2ms/step - loss: 4316.0596 - mean_squared_error: 4316.0596
Epoch 8/30
36/36 [==============================] - 0s 2ms/ste

In [1]:


# Load and preprocess the dataset
faces = fetch_olivetti_faces()

X = faces.images
y = to_categorical(faces.target)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = X_train.reshape((-1, 64*64))
X_test = X_test.reshape((-1, 64*64))

min_max_scaler = MinMaxScaler(feature_range=(0.01, 0.99))

# Fit and transform the training data
X_train_scaled = min_max_scaler.fit_transform(X_train)

# Transform the test data using the same scaler
X_test_scaled = min_max_scaler.transform(X_test)

num_features = X_train.shape[1]
num_classes = y.shape[1]

print(num_classes)

batch_size = 10
epochs = 6

# # # AutoKeras model
# # ak_model = ak.StructuredDataClassifier(max_trials=5, overwrite=True)

# # # Train the AutoKeras model
# # ak_model.fit(X_train, y_train, epochs=epochs)

# # # Evaluate AutoKeras model
# # loss, ak_accuracy = ak_model.evaluate(X_test, y_test)
# # print(f"AutoKeras Model Accuracy: {ak_accuracy}")

print('******************************************************************************************')
print('******************************************************************************************')
print('******************************************************************************************')
print('******************************************************************************************')
print('******************************************************************************************')

# Create and train the model
model = create_optimized_model(num_features, 3000 , num_classes, 'accuracy',0.00005)


# Train the model
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=1)

# Evaluate the model

loss, accuracy = model.evaluate(X_test, y_test)
print(f"Accuracy: {accuracy}")



NameError: ignored

In [28]:
print(X_test[1])

[0.18595041 0.12809917 0.11570248 ... 0.19008264 0.2107438  0.2107438 ]


In [ ]:


# Load and preprocess the dataset
linnerud = load_linnerud()
X = linnerud.data
y = linnerud.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Using MinMaxScaler to scale the data between 0 and 1
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

num_features = X_train.shape[1]
# num_classes = y_train.shape[1]

print(num_features)

# Create and train the model
model = create_optimized_model(num_features, 20, 1, 'mae')
batch_size = 10
epochs = 100

# Train the model
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=1)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Accuracy: {accuracy}")

3
Epoch 1/100
2/2 [==============================] - 2s 12ms/step - loss: 12236.7773 - mae: 90.0593
Epoch 2/100
2/2 [==============================] - 0s 10ms/step - loss: 12203.7246 - mae: 89.8693
Epoch 3/100
2/2 [==============================] - 0s 9ms/step - loss: 12176.3438 - mae: 89.7125
Epoch 4/100
2/2 [==============================] - 0s 11ms/step - loss: 12145.0156 - mae: 89.5329
Epoch 5/100
2/2 [==============================] - 0s 8ms/step - loss: 12115.8711 - mae: 89.3676
Epoch 6/100
2/2 [==============================] - 0s 13ms/step - loss: 12086.8057 - mae: 89.1992
Epoch 7/100
2/2 [==============================] - 0s 7ms/step - loss: 12056.2979 - mae: 89.0207
Epoch 8/100
2/2 [==============================] - 0s 7ms/step - loss: 12025.0195 - mae: 88.8407
Epoch 9/100
2/2 [==============================] - 0s 7ms/step - loss: 11993.6465 - mae: 88.6577
Epoch 10/100
2/2 [==============================] - 0s 18ms/step - loss: 11962.9824 - mae: 88.4801
Epoch 11/100
2/2 [====